# 24hr Day-Ahead API Forecast Demo

This notebook walks through the Ona Freemium API workflow and visualises the 24-hour forecast output.

- **Load** your site's CSV (or use the bundled Sibaya sample)
- **Submit** via the Freemium API (email verify + CSV upload)
- **Visualise** historical actuals alongside the 24-hour forecast as a dual bar chart

No AWS credentials required — uses the public API only.

In [ ]:
import io
import json
import os

import matplotlib.dates as mdates
import matplotlib.pyplot as plt
import pandas as pd
import requests

plt.style.use('seaborn-v0_8-whitegrid')

# ---- Configuration ----
API_BASE_URL = os.getenv("API_BASE_URL", "https://api.asoba.co/api/v1")
DEMO_EMAIL = os.getenv("DEMO_EMAIL", "")          # your email for verification
CSV_FILE_PATH = os.getenv("CSV_FILE_PATH", "")     # path to your CSV, or leave blank for bundled sample

# Freemium submission metadata
SITE_NAME = os.getenv("FREEMIUM_SITE_NAME", "Sibaya Live Demo")
LOCATION = os.getenv("FREEMIUM_LOCATION", "Durban")

print("API_BASE_URL:", API_BASE_URL)
if DEMO_EMAIL:
    print("DEMO_EMAIL:", DEMO_EMAIL)
else:
    print("\u26a0\ufe0f  Set DEMO_EMAIL before running the API cells (env var or edit above).")

## Step 1 — Load your data

Set `CSV_FILE_PATH` above to point at your own CSV, or leave it blank to use the
bundled Sibaya sample (`sample_sibaya_freemium.csv`).

Minimum CSV requirements:
- `timestamp` column (ISO 8601)
- `kWh` column (numeric, hourly energy production)

> **ODS-E note:** You do _not_ need to pre-process or reformat the CSV before uploading.
> The Freemium API auto-detects the inverter manufacturer from column names
> (Huawei, Enphase, Solarman, Telkom, Lux, Switch, Utility, etc.) and converts
> the raw OEM export to the ODS-E `production_timeseries` schema
> (`timestamp`, `kWh`, `error_type`) server-side.
> The bundled Sibaya sample is already in ODS-E format; a prospect's raw inverter
> export will be converted automatically on upload.

In [ ]:
SAMPLE_CSV = os.path.join(os.path.dirname(os.path.abspath("__file__")), "sample_sibaya_freemium.csv")

csv_path = CSV_FILE_PATH if CSV_FILE_PATH else SAMPLE_CSV
print("Loading CSV:", csv_path)

upload_df = pd.read_csv(csv_path)
upload_df["timestamp"] = pd.to_datetime(upload_df["timestamp"], utc=True)
upload_df["kWh"] = pd.to_numeric(upload_df["kWh"], errors="coerce")

print(f"Rows: {len(upload_df)}")
print(f"Date range: {upload_df['timestamp'].min()} \u2192 {upload_df['timestamp'].max()}")
print(f"Mean kWh: {upload_df['kWh'].mean():.2f}")
print()
display(upload_df.head())
display(upload_df.describe())

## Step 2 — Email verification

Run the cell below to send a verification code to `DEMO_EMAIL`.
Check your inbox for the 6-digit code.

In [ ]:
if not DEMO_EMAIL:
    raise RuntimeError("Set DEMO_EMAIL first (env var or edit the config cell).")

verify_resp = requests.post(
    f"{API_BASE_URL}/freemium-forecast/verify",
    headers={"Content-Type": "application/json"},
    data=json.dumps({"email": DEMO_EMAIL}),
    timeout=30,
)

print("Status:", verify_resp.status_code)
print(json.dumps(verify_resp.json(), indent=2))

## Step 3 — Submit forecast

Paste the 6-digit verification code into `VERIFICATION_CODE` below, then run the cell
to upload the CSV and receive a 24-hour forecast.

The API standardises the uploaded CSV to ODS-E before generating the forecast, so the
`manufacturer` field in the response reflects the inverter brand that was auto-detected
from the CSV column names.

In [ ]:
VERIFICATION_CODE = os.getenv("VERIFICATION_CODE", "")  # or set directly: "123456"
if not VERIFICATION_CODE:
    raise RuntimeError("Set VERIFICATION_CODE (env var or edit this cell) with the 6-digit code from your email.")

# Build CSV bytes from the loaded dataframe
csv_bytes = upload_df.to_csv(index=False).encode("utf-8")

forecast_resp = requests.post(
    f"{API_BASE_URL}/freemium-forecast",
    files={"file": ("upload.csv", io.BytesIO(csv_bytes), "text/csv")},
    data={
        "email": DEMO_EMAIL,
        "verification_code": VERIFICATION_CODE,
        "site_name": SITE_NAME,
        "location": LOCATION,
    },
    timeout=120,
)

print("Status:", forecast_resp.status_code)
freemium_result = forecast_resp.json()

if freemium_result.get("status") == "success":
    fc = freemium_result.get("forecast", {})
    points = fc.get("forecasts", [])
    print("\n\u2705 Forecast generated")
    print("Customer ID:", freemium_result.get("customer_id"))
    print("Manufacturer:", fc.get("manufacturer"))
    print("Model type:", fc.get("model_type"))
    print("Remaining forecasts this month:", freemium_result.get("remaining_forecasts"))
    print("Forecast points:", len(points))
    if points:
        display(pd.DataFrame(points).head())
else:
    print("\u274c Forecast failed")
    print(json.dumps(freemium_result, indent=2))

## Step 4 — Visualise results

The chart below shows the last 24 hours of historical actuals from your CSV (blue)
alongside the 24-hour forecast output (orange).

In [ ]:
# --- Parse actuals (last 24h of uploaded CSV) ---
actuals = upload_df.sort_values("timestamp").tail(24).copy()
actuals = actuals.set_index("timestamp")

# --- Parse forecast points ---
fc_data = freemium_result.get("forecast", {})
forecast_points = fc_data.get("forecasts", [])
forecast_df = pd.DataFrame(forecast_points)
forecast_df["timestamp"] = pd.to_datetime(forecast_df["timestamp"], utc=True)
forecast_df["kWh_forecast"] = pd.to_numeric(forecast_df["kWh_forecast"], errors="coerce")
forecast_df = forecast_df.sort_values("timestamp").set_index("timestamp")

# --- Dual bar chart ---
fig, ax = plt.subplots(figsize=(14, 5))

bar_width = pd.Timedelta(minutes=25)

ax.bar(
    actuals.index,
    actuals["kWh"],
    width=bar_width,
    color="#3182bd",
    alpha=0.85,
    label="Historical actuals",
)
ax.bar(
    forecast_df.index,
    forecast_df["kWh_forecast"],
    width=bar_width,
    color="#e6550d",
    alpha=0.85,
    label="24h forecast",
)

# Annotate peak forecast hour
if not forecast_df.empty:
    peak_idx = forecast_df["kWh_forecast"].idxmax()
    peak_val = forecast_df.loc[peak_idx, "kWh_forecast"]
    ax.annotate(
        f"Peak: {peak_val:.1f} kWh",
        xy=(peak_idx, peak_val),
        xytext=(0, 12),
        textcoords="offset points",
        ha="center",
        fontsize=9,
        fontweight="bold",
        color="#e6550d",
        arrowprops=dict(arrowstyle="->", color="#e6550d", lw=1.2),
    )

site_label = fc_data.get("site_name", SITE_NAME)
ax.set_title(f"{site_label} \u2014 Historical vs 24h Forecast", fontsize=14, fontweight="bold")
ax.set_ylabel("kWh")
ax.set_xlabel("Time (UTC)")
ax.xaxis.set_major_locator(mdates.HourLocator(interval=3))
ax.xaxis.set_major_formatter(mdates.DateFormatter("%m/%d %H:%M"))
fig.autofmt_xdate(rotation=45)
ax.legend(loc="upper left")
ax.grid(axis="y", alpha=0.4)

plt.tight_layout()
plt.show()

## Summary & next steps

What you just saw:
1. Your CSV was uploaded to the Ona platform via the public Freemium API.
2. The platform auto-detected your inverter manufacturer and standardised the data.
3. A 24-hour ahead forecast was generated and returned in seconds.

**Same pipeline, your data, your site.**

To run this with your own data, change three values at the top of the notebook:
- `CSV_FILE_PATH` — path to your site's CSV
- `DEMO_EMAIL` — your email address
- `VERIFICATION_CODE` — the code sent to your email